In [2]:
import csv

input_file = "tv_shows_temp.csv"
output_file = "tv_shows_temp_fixed.csv"

with open(input_file, "r", newline="", encoding="utf-8") as infile, \
     open(output_file, "w", newline="", encoding="utf-8") as outfile:
    reader = csv.reader(infile)
    # Use QUOTE_ALL to force quoting of every field
    writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
    for row in reader:
        writer.writerow(row)

print("CSV file has been rewritten as:", output_file)


CSV file has been rewritten as: tv_shows_temp_fixed.csv


In [3]:
import csv

input_file = "tv_shows_temp_fixed.csv"
output_file = "tv_shows_temp_fixed_null.csv"

with open(input_file, "r", newline="", encoding="utf-8") as infile, \
     open(output_file, "w", newline="", encoding="utf-8") as outfile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames
    writer = csv.DictWriter(outfile, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    for row in reader:
        # Replace empty first_air_date with \N
        if row.get("first_air_date", "").strip() == "":
            row["first_air_date"] = r"\N"
        writer.writerow(row)

print("CSV fixed; output written to", output_file)


CSV fixed; output written to tv_shows_temp_fixed_null.csv
